In [1]:
# ============================================================================
# RQ1: WHEN AND TO WHAT EXTENT DOES META'S POLICY AFFECT REACH?
# Data-Driven Breakpoint Analysis (Cross-Algorithm Validated)
# VERSION 4: Updated Table/Figure Numbering to Match Working Paper
# ============================================================================
# 
# RESEARCH DESIGN:
# ----------------
# RQ1: When and to what extent did Meta's political content reduction policy—
#      and its subsequent reversal—affect political actors' reach on Facebook 
#      in Italy?
#
# METHODOLOGY: Cross-Algorithm Validated Breakpoint Detection
# -----------------------------------------------------------
# This analysis uses a SINGLE, CONSISTENT breakpoint identification approach:
#
#   STEP 1 - DETECTION:
#     • Run Bai-Perron structural break detection on 4 metrics (views, reactions,
#       shares, comments)
#     • Run PELT changepoint detection on the same 4 metrics
#     • Total: Up to 8 possible detections per method cluster
#
#   STEP 2 - CLUSTERING:
#     • Group detected dates within a 30-day tolerance window
#     • Calculate consensus date (median) and detection spread (range)
#
#   STEP 3 - CROSS-VALIDATION:
#     • Retain only breakpoints detected by BOTH algorithms (Bai-Perron AND PELT)
#     • This ensures statistical robustness across methodologies
#
#   STEP 4 - FINAL SELECTION (Three-Breakpoint Model):
#     When ≥3 cross-validated breakpoints exist:
#       • T1: First chronological breakpoint (Policy Implementation)
#       • T3: First breakpoint after Sept 2024 OR last chronological (Reversal)
#       • T2: Among remaining intermediate breakpoints, select the one with
#             MOST method detections (strongest evidence); ties broken by date
#     When 2 breakpoints: T1 + T3 only (no T2)
#     When 1 breakpoint: T1 only
#
# WORKING PAPER TABLE/FIGURE MAPPING:
# -----------------------------------
# Table 1:  Political Actor Groups (descriptive - not generated here)
# Table 2:  Account and Post Counts by Group
# Table 3:  Weekly Aggregated Engagement Statistics
# Table 4:  Cross-Validated Breakpoints
# Table 5:  Reach Statistics by Policy Phase (Re-elected MPs)
# Table 6:  Engagement Metrics by Policy Phase (Re-elected MPs)
# Table 7:  Breakpoint Validation Across Groups
# Table 8:  Cross-Group Magnitude Comparison (Views)
# Table 9:  Pairwise Phase Comparisons (Dunn's Test)
# Table 10: Per-Post vs Total Weekly Reach by Group
#
# Figure 1: Time series of all groups with breakpoints
# Figure 2: MPs Reelected trends with breakpoints
# Figure 3: Individual group trends (faceted validation)
#
# Discovery Sample: Re-elected MPs (continuous presence 2020-2025)
# Validation Groups: New MPs, Prominent Politicians, Extremists
#
# Dataset: weekly_aggregation (as per DATASETS_QUICK_REFERENCE.md)
# ============================================================================

# Required packages
required_packages <- c(
  "tidyverse", "lubridate", "strucchange", "changepoint", 
  "zoo", "segmented", "patchwork", "scales", "knitr", "moments"
)

for (pkg in required_packages) {
  if (!require(pkg, character.only = TRUE, quietly = TRUE)) {
    install.packages(pkg, dependencies = TRUE)
  }
  library(pkg, character.only = TRUE)
}

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("RQ1 ANALYSIS: META'S POLITICAL CONTENT POLICY EFFECTS IN ITALY\n")
cat("Cross-Algorithm Validated Breakpoint Detection\n")
cat("Version 4: Table/Figure Numbering Aligned with Working Paper\n")
cat(rep("=", 80), "\n\n", sep = "")

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================

cat("STEP 1: LOADING DATA\n")
cat(rep("-", 40), "\n\n", sep = "")

# Find the most recent weekly aggregation file
weekly_files <- list.files(
  path = "cleaned_data",
  pattern = "weekly_aggregation_.*\\.rds$",
  full.names = TRUE
)

if (length(weekly_files) == 0) {
  stop("No weekly_aggregation files found in cleaned_data/")
}

weekly_file <- sort(weekly_files, decreasing = TRUE)[1]
cat("Loading weekly data:", weekly_file, "\n")
weekly_data <- readRDS(weekly_file)

cat("\nData structure:\n")
cat("  Rows:", nrow(weekly_data), "\n")
cat("  Columns:", ncol(weekly_data), "\n")
cat("  Date range:", as.character(min(weekly_data$week)), "to", 
    as.character(max(weekly_data$week)), "\n\n")

# Detect dataset version (3-group vs 4-group)
mp_groups <- unique(weekly_data$main_list[grepl("^MPs", weekly_data$main_list)])
all_groups <- unique(weekly_data$main_list)

dataset_version <- if (length(mp_groups) > 1) "v3.2_split" else "v2_v3.1"

# Identify discovery and validation samples
discovery_group <- if ("MPs_Reelected" %in% all_groups) "MPs_Reelected" else "MPs"
validation_groups <- setdiff(all_groups, discovery_group)

cat("Dataset Configuration:\n")
cat("  Version:", dataset_version, "\n")
cat("  Discovery sample:", discovery_group, "\n")
cat("  Validation groups:", paste(validation_groups, collapse = ", "), "\n\n")

# ============================================================================
# TABLE 2: ACCOUNT AND POST COUNTS BY GROUP
# (Working Paper Table 2)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 2: ACCOUNT AND POST COUNTS BY GROUP\n")
cat("(Working Paper Table 2)\n")
cat(rep("=", 80), "\n\n", sep = "")

# Load accounts summary for n_accounts if available
accounts_files <- list.files(
  path = "cleaned_data",
  pattern = "accounts_summary_.*\\.rds$",
  full.names = TRUE
)

# Calculate Table 2 statistics from weekly data
table2_from_weekly <- weekly_data %>%
  group_by(main_list) %>%
  summarise(
    n_weeks = n(),
    total_posts = sum(n_posts, na.rm = TRUE),
    total_views = sum(total_views, na.rm = TRUE),
    avg_views_overall = mean(avg_views, na.rm = TRUE),
    .groups = "drop"
  )

# Try to get account counts from accounts_summary
if (length(accounts_files) > 0) {
  accounts_summary <- readRDS(sort(accounts_files, decreasing = TRUE)[1])
  
  account_counts <- accounts_summary %>%
    group_by(main_list) %>%
    summarise(
      n_accounts = n(),
      .groups = "drop"
    )
  
  table2_from_weekly <- table2_from_weekly %>%
    left_join(account_counts, by = "main_list")
} else {
  if ("n_accounts" %in% names(weekly_data)) {
    account_counts <- weekly_data %>%
      group_by(main_list) %>%
      summarise(
        n_accounts = max(n_accounts, na.rm = TRUE),
        .groups = "drop"
      )
    table2_from_weekly <- table2_from_weekly %>%
      left_join(account_counts, by = "main_list")
  } else {
    table2_from_weekly$n_accounts <- NA
  }
}

# Format Table 2 for working paper
table2 <- data.frame(
  Group = table2_from_weekly$main_list,
  N_Accounts = ifelse(is.na(table2_from_weekly$n_accounts), NA, table2_from_weekly$n_accounts),
  Total_Posts = table2_from_weekly$total_posts,
  Avg_Views = round(table2_from_weekly$avg_views_overall, 0),
  stringsAsFactors = FALSE
)

cat("TABLE 2 - Account and Post Counts by Group:\n\n")
print(table2, row.names = FALSE)
cat("\n")

# Save Table 2
write.csv(table2, "RQ1_Table2_account_post_counts.csv", row.names = FALSE)
cat("Saved: RQ1_Table2_account_post_counts.csv\n\n")

# ============================================================================
# TABLE 3: WEEKLY AGGREGATED ENGAGEMENT STATISTICS
# (Working Paper Table 3)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 3: WEEKLY AGGREGATED ENGAGEMENT STATISTICS\n")
cat("(Working Paper Table 3)\n")
cat(rep("=", 80), "\n\n", sep = "")

# Calculate comprehensive engagement statistics by group
table3_stats <- weekly_data %>%
  group_by(main_list) %>%
  summarise(
    # Views
    views_mean = mean(avg_views, na.rm = TRUE),
    views_sd = sd(avg_views, na.rm = TRUE),
    views_median = median(avg_views, na.rm = TRUE),
    views_min = min(avg_views, na.rm = TRUE),
    views_max = max(avg_views, na.rm = TRUE),
    # Reactions
    reactions_mean = mean(avg_reactions, na.rm = TRUE),
    reactions_sd = sd(avg_reactions, na.rm = TRUE),
    reactions_median = median(avg_reactions, na.rm = TRUE),
    reactions_min = min(avg_reactions, na.rm = TRUE),
    reactions_max = max(avg_reactions, na.rm = TRUE),
    # Shares
    shares_mean = mean(avg_shares, na.rm = TRUE),
    shares_sd = sd(avg_shares, na.rm = TRUE),
    shares_median = median(avg_shares, na.rm = TRUE),
    shares_min = min(avg_shares, na.rm = TRUE),
    shares_max = max(avg_shares, na.rm = TRUE),
    # Comments
    comments_mean = mean(avg_comments, na.rm = TRUE),
    comments_sd = sd(avg_comments, na.rm = TRUE),
    comments_median = median(avg_comments, na.rm = TRUE),
    comments_min = min(avg_comments, na.rm = TRUE),
    comments_max = max(avg_comments, na.rm = TRUE),
    .groups = "drop"
  )

# Display Views
cat("Views (Reach):\n")
table3_views <- table3_stats %>%
  transmute(
    Group = main_list,
    Mean = round(views_mean, 1),
    SD = round(views_sd, 1),
    Median = round(views_median, 1),
    Min = round(views_min, 1),
    Max = round(views_max, 1)
  )
print(as.data.frame(table3_views), row.names = FALSE)
cat("\n")

# Display Reactions
cat("Reactions:\n")
table3_reactions <- table3_stats %>%
  transmute(
    Group = main_list,
    Mean = round(reactions_mean, 1),
    SD = round(reactions_sd, 1),
    Median = round(reactions_median, 1),
    Min = round(reactions_min, 1),
    Max = round(reactions_max, 1)
  )
print(as.data.frame(table3_reactions), row.names = FALSE)
cat("\n")

# Display Shares
cat("Shares:\n")
table3_shares <- table3_stats %>%
  transmute(
    Group = main_list,
    Mean = round(shares_mean, 1),
    SD = round(shares_sd, 1),
    Median = round(shares_median, 1),
    Min = round(shares_min, 1),
    Max = round(shares_max, 1)
  )
print(as.data.frame(table3_shares), row.names = FALSE)
cat("\n")

# Display Comments
cat("Comments:\n")
table3_comments <- table3_stats %>%
  transmute(
    Group = main_list,
    Mean = round(comments_mean, 1),
    SD = round(comments_sd, 1),
    Median = round(comments_median, 1),
    Min = round(comments_min, 1),
    Max = round(comments_max, 1)
  )
print(as.data.frame(table3_comments), row.names = FALSE)
cat("\n")

# Save Table 3 as CSV (long format)
table3_long <- table3_stats %>%
  pivot_longer(
    cols = -main_list,
    names_to = c("metric", "stat"),
    names_sep = "_",
    values_to = "value"
  ) %>%
  pivot_wider(
    names_from = stat,
    values_from = value
  ) %>%
  rename(Group = main_list, Metric = metric)

write.csv(table3_long, "RQ1_Table3_engagement_stats.csv", row.names = FALSE)
cat("Saved: RQ1_Table3_engagement_stats.csv\n\n")

# ============================================================================
# STEP 2: KEY DATES REFERENCE
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("STEP 2: KEY DATES REFERENCE\n")
cat(rep("=", 80), "\n\n", sep = "")

# Meta policy timeline
meta_policy_dates <- data.frame(
  date = as.Date(c(
    "2021-02-10", "2022-07-19", "2023-04-20", "2025-01-07"
  )),
  event = c(
    "Initial announcement",
    "Global implementation",
    "Refinements (survey-based signals)",
    "Policy reversal"
  ),
  stringsAsFactors = FALSE
)

# Electoral events
italian_election_date <- as.Date("2022-09-25")
italian_election_window_start <- as.Date("2022-08-01")
italian_election_window_end <- as.Date("2022-11-30")

eu_election_date <- as.Date("2024-06-09")
eu_election_window_start <- as.Date("2024-05-01")
eu_election_window_end <- as.Date("2024-07-31")

election_events <- data.frame(
  election = c("Italian General Election 2022", "EU Parliamentary Election 2024"),
  date = c(italian_election_date, eu_election_date),
  window_start = c(italian_election_window_start, eu_election_window_start),
  window_end = c(italian_election_window_end, eu_election_window_end),
  stringsAsFactors = FALSE
)

cat("META POLICY TIMELINE:\n")
for (i in 1:nrow(meta_policy_dates)) {
  cat("  ", as.character(meta_policy_dates$date[i]), " - ", 
      meta_policy_dates$event[i], "\n", sep = "")
}
cat("\n")

cat("ELECTORAL EVENTS:\n")
cat("  Italian General Election 2022:", as.character(italian_election_date), "\n")
cat("  EU Parliamentary Election 2024:", as.character(eu_election_date), "\n\n")

# ============================================================================
# STEP 3: BREAKPOINT DETECTION (Discovery Sample)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("STEP 3: BREAKPOINT DETECTION (Discovery Sample)\n")
cat("Discovery sample:", discovery_group, "\n")
cat(rep("=", 80), "\n\n", sep = "")

# Extract discovery sample
discovery_data <- weekly_data %>%
  filter(main_list == discovery_group) %>%
  arrange(week) %>%
  mutate(time_index = row_number())

cat("Discovery sample size:", nrow(discovery_data), "weeks\n\n")

# -----------------------------------------------------------------------------
# 3.1 Bai-Perron Detection
# -----------------------------------------------------------------------------

cat("3.1 BAI-PERRON STRUCTURAL BREAK DETECTION\n")
cat(rep("-", 60), "\n\n", sep = "")

detect_bai_perron <- function(data, metric, max_breaks = 3) {
  
  clean_data <- data %>%
    filter(!is.na(.data[[metric]])) %>%
    mutate(time_index = row_number())
  
  bp_result <- tryCatch({
    breakpoints(
      as.formula(paste(metric, "~ 1")),
      data = clean_data,
      h = 0.15,
      breaks = max_breaks
    )
  }, error = function(e) {
    cat("  Error for", metric, ":", e$message, "\n")
    return(NULL)
  })
  
  if (is.null(bp_result) || is.na(bp_result$breakpoints[1])) {
    return(NULL)
  }
  
  break_indices <- bp_result$breakpoints
  break_dates <- clean_data$week[break_indices]
  
  cat("  ", metric, ": ", length(break_dates), " breakpoints detected\n", sep = "")
  for (i in seq_along(break_dates)) {
    cat("    Break", i, ":", as.character(break_dates[i]), "\n")
  }
  
  return(list(
    dates = break_dates,
    indices = break_indices,
    metric = metric,
    algorithm = "Bai-Perron"
  ))
}

# Run Bai-Perron on all metrics
bp_results <- list()
for (metric in c("avg_views", "avg_reactions", "avg_shares", "avg_comments")) {
  bp_results[[metric]] <- detect_bai_perron(discovery_data, metric)
}
cat("\n")

# -----------------------------------------------------------------------------
# 3.2 PELT Detection
# -----------------------------------------------------------------------------

cat("3.2 PELT CHANGEPOINT DETECTION\n")
cat(rep("-", 60), "\n\n", sep = "")

detect_pelt <- function(data, metric) {
  
  clean_data <- data %>%
    filter(!is.na(.data[[metric]]))
  
  values <- clean_data[[metric]]
  
  pelt_result <- tryCatch({
    cpt.meanvar(values, method = "PELT", penalty = "BIC")
  }, error = function(e) {
    cat("  Error for", metric, ":", e$message, "\n")
    return(NULL)
  })
  
  if (is.null(pelt_result)) return(NULL)
  
  changepoints <- cpts(pelt_result)
  
  if (length(changepoints) == 0) {
    cat("  ", metric, ": No changepoints detected\n", sep = "")
    return(NULL)
  }
  
  change_dates <- clean_data$week[changepoints]
  
  cat("  ", metric, ": ", length(change_dates), " changepoints detected\n", sep = "")
  for (i in seq_along(change_dates)) {
    cat("    Changepoint", i, ":", as.character(change_dates[i]), "\n")
  }
  
  return(list(
    dates = change_dates,
    indices = changepoints,
    metric = metric,
    algorithm = "PELT"
  ))
}

# Run PELT on all metrics
pelt_results <- list()
for (metric in c("avg_views", "avg_reactions", "avg_shares", "avg_comments")) {
  pelt_results[[metric]] <- detect_pelt(discovery_data, metric)
}
cat("\n")

# -----------------------------------------------------------------------------
# 3.3 Build Consensus
# -----------------------------------------------------------------------------

cat("3.3 BUILDING CONSENSUS BREAKPOINTS\n")
cat(rep("-", 60), "\n\n", sep = "")

build_consensus_breakpoints <- function(bp_list, pelt_list, tolerance_days = 30) {
  
  all_detections <- data.frame()
  
  for (metric in names(bp_list)) {
    if (!is.null(bp_list[[metric]])) {
      all_detections <- rbind(all_detections, data.frame(
        date = bp_list[[metric]]$dates,
        metric = metric,
        algorithm = "Bai-Perron",
        stringsAsFactors = FALSE
      ))
    }
  }
  
  for (metric in names(pelt_list)) {
    if (!is.null(pelt_list[[metric]])) {
      all_detections <- rbind(all_detections, data.frame(
        date = pelt_list[[metric]]$dates,
        metric = metric,
        algorithm = "PELT",
        stringsAsFactors = FALSE
      ))
    }
  }
  
  if (nrow(all_detections) == 0) {
    cat("No breakpoints detected by any method.\n")
    return(NULL)
  }
  
  all_detections <- all_detections %>%
    mutate(date = as.Date(date)) %>%
    arrange(date)
  
  cat("Total detections:", nrow(all_detections), "\n\n")
  
  # Cluster nearby dates
  all_detections <- all_detections %>%
    mutate(cluster = cumsum(c(1, diff(date) > tolerance_days)))
  
  # Summarize clusters
  consensus <- all_detections %>%
    group_by(cluster) %>%
    summarise(
      consensus_date = median(date),
      date_min = min(date),
      date_max = max(date),
      date_range_days = as.numeric(max(date) - min(date)),
      n_methods = n(),
      n_algorithms = n_distinct(algorithm),
      has_bai_perron = any(algorithm == "Bai-Perron"),
      has_pelt = any(algorithm == "PELT"),
      metrics_bp = paste(unique(metric[algorithm == "Bai-Perron"]), collapse = ", "),
      metrics_pelt = paste(unique(metric[algorithm == "PELT"]), collapse = ", "),
      .groups = "drop"
    ) %>%
    mutate(
      cross_validated = n_algorithms >= 2,
      strength = case_when(
        cross_validated & n_methods >= 6 ~ "VERY STRONG",
        cross_validated & n_methods >= 4 ~ "STRONG",
        cross_validated ~ "MODERATE",
        n_methods >= 4 ~ "Single-Algo (Many)",
        TRUE ~ "WEAK"
      ),
      date_range = ifelse(
        date_min == date_max,
        as.character(date_min),
        paste(date_min, "to", date_max)
      )
    ) %>%
    arrange(consensus_date)
  
  cat("CONSENSUS BREAKPOINTS:\n\n")
  for (i in 1:nrow(consensus)) {
    cat("Cluster", i, ":\n")
    cat("  Consensus date:", as.character(consensus$consensus_date[i]), "\n")
    cat("  Detection range:", consensus$date_range[i], "\n")
    cat("  Methods:", consensus$n_methods[i], "| Algorithms:", consensus$n_algorithms[i], "\n")
    cat("  Cross-Validated:", ifelse(consensus$cross_validated[i], "YES", "NO"), "\n")
    cat("  Strength:", consensus$strength[i], "\n\n")
  }
  
  return(consensus)
}

consensus_breakpoints <- build_consensus_breakpoints(bp_results, pelt_results)

# ============================================================================
# TABLE 4: CROSS-VALIDATED BREAKPOINTS
# (Working Paper Table 4)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 4: CROSS-VALIDATED BREAKPOINTS\n")
cat("(Working Paper Table 4)\n")
cat(rep("=", 80), "\n\n", sep = "")

# Get cross-validated breakpoints
cross_validated <- consensus_breakpoints %>%
  filter(cross_validated == TRUE) %>%
  arrange(consensus_date)

if (nrow(cross_validated) == 0) {
  cat("WARNING: No cross-validated breakpoints found!\n")
  cross_validated <- consensus_breakpoints %>%
    filter(n_methods >= 4) %>%
    arrange(consensus_date)
}

# Assign final breakpoints (T1, T2, T3)
assign_final_breakpoints <- function(cv_bp) {
  
  n_bp <- nrow(cv_bp)
  
  if (n_bp == 0) {
    stop("No breakpoints available for analysis!")
  }
  
  create_bp_struct <- function(row) {
    if (is.null(row) || nrow(row) == 0) return(NULL)
    list(
      date = as.Date(row$consensus_date),
      date_min = as.Date(row$date_min),
      date_max = as.Date(row$date_max),
      date_range_days = row$date_range_days,
      methods = row$n_methods,
      n_algorithms = row$n_algorithms,
      cross_validated = row$cross_validated,
      strength = row$strength,
      date_range = row$date_range
    )
  }
  
  breakpoints <- list()
  selected_indices <- c()
  
  if (n_bp >= 1) {
    breakpoints$T1 <- create_bp_struct(cv_bp[1, ])
    selected_indices <- c(selected_indices, 1)
  }
  
  if (n_bp >= 3) {
    reversal_candidates <- cv_bp %>% 
      mutate(row_idx = row_number()) %>%
      filter(consensus_date >= as.Date("2024-09-01"))
    
    if (nrow(reversal_candidates) > 0) {
      selected_idx <- reversal_candidates$row_idx[1]
      breakpoints$T3 <- create_bp_struct(cv_bp[selected_idx, ])
      selected_indices <- c(selected_indices, selected_idx)
    } else {
      breakpoints$T3 <- create_bp_struct(cv_bp[n_bp, ])
      selected_indices <- c(selected_indices, n_bp)
    }
    
    middle_candidates <- cv_bp %>%
      mutate(row_idx = row_number()) %>%
      filter(consensus_date > breakpoints$T1$date,
             consensus_date < breakpoints$T3$date) %>%
      arrange(desc(n_methods), consensus_date)
    
    if (nrow(middle_candidates) > 0) {
      selected_idx <- middle_candidates$row_idx[1]
      breakpoints$T2 <- create_bp_struct(cv_bp[selected_idx, ])
      selected_indices <- c(selected_indices, selected_idx)
    } else {
      breakpoints$T2 <- NULL
    }
    
  } else if (n_bp == 2) {
    breakpoints$T3 <- create_bp_struct(cv_bp[2, ])
    selected_indices <- c(selected_indices, 2)
    breakpoints$T2 <- NULL
  } else {
    breakpoints$T2 <- NULL
    breakpoints$T3 <- NULL
  }
  
  return(breakpoints)
}

FINAL_BREAKPOINTS <- assign_final_breakpoints(cross_validated)

# Create Table 4 data frame
table4_data <- data.frame(
  Breakpoint = character(),
  Point_Estimate = character(),
  Methods = numeric(),
  Strength = character(),
  Detection_Range = character(),
  stringsAsFactors = FALSE
)

if (!is.null(FINAL_BREAKPOINTS$T1)) {
  table4_data <- rbind(table4_data, data.frame(
    Breakpoint = "T1 (Implementation)",
    Point_Estimate = as.character(FINAL_BREAKPOINTS$T1$date),
    Methods = FINAL_BREAKPOINTS$T1$methods,
    Strength = FINAL_BREAKPOINTS$T1$strength,
    Detection_Range = FINAL_BREAKPOINTS$T1$date_range,
    stringsAsFactors = FALSE
  ))
}

if (!is.null(FINAL_BREAKPOINTS$T2)) {
  table4_data <- rbind(table4_data, data.frame(
    Breakpoint = "T2 (Adjustment)",
    Point_Estimate = as.character(FINAL_BREAKPOINTS$T2$date),
    Methods = FINAL_BREAKPOINTS$T2$methods,
    Strength = FINAL_BREAKPOINTS$T2$strength,
    Detection_Range = FINAL_BREAKPOINTS$T2$date_range,
    stringsAsFactors = FALSE
  ))
}

if (!is.null(FINAL_BREAKPOINTS$T3)) {
  table4_data <- rbind(table4_data, data.frame(
    Breakpoint = "T3 (Reversal)",
    Point_Estimate = as.character(FINAL_BREAKPOINTS$T3$date),
    Methods = FINAL_BREAKPOINTS$T3$methods,
    Strength = FINAL_BREAKPOINTS$T3$strength,
    Detection_Range = FINAL_BREAKPOINTS$T3$date_range,
    stringsAsFactors = FALSE
  ))
}

cat("TABLE 4 - Cross-Validated Breakpoints:\n\n")
print(table4_data, row.names = FALSE)
cat("\n")

write.csv(table4_data, "RQ1_Table4_breakpoints.csv", row.names = FALSE)
cat("Saved: RQ1_Table4_breakpoints.csv\n\n")

# Determine model type
MODEL_TYPE <- case_when(
  !is.null(FINAL_BREAKPOINTS$T2) ~ "THREE_BREAKPOINT",
  !is.null(FINAL_BREAKPOINTS$T3) ~ "TWO_BREAKPOINT",
  TRUE ~ "ONE_BREAKPOINT"
)

cat("MODEL TYPE:", MODEL_TYPE, "\n\n")

# ============================================================================
# STEP 4: APPLY PHASES TO ALL DATA
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("STEP 4: APPLY PHASES TO ALL DATA\n")
cat(rep("=", 80), "\n\n", sep = "")

apply_phases <- function(data, breakpoints, model_type) {
  
  T1 <- breakpoints$T1$date
  T2 <- if (!is.null(breakpoints$T2)) breakpoints$T2$date else NULL
  T3 <- if (!is.null(breakpoints$T3)) breakpoints$T3$date else NULL
  
  if (model_type == "THREE_BREAKPOINT") {
    data <- data %>%
      mutate(
        phase = case_when(
          week < T1 ~ "0_Pre-Policy",
          week >= T1 & week < T2 ~ "1_Policy-Active",
          week >= T2 & week < T3 ~ "2_Adjusted-Policy",
          week >= T3 ~ "3_Post-Reversal"
        ),
        phase = factor(phase, levels = c(
          "0_Pre-Policy", "1_Policy-Active", "2_Adjusted-Policy", "3_Post-Reversal"
        ))
      )
  } else if (model_type == "TWO_BREAKPOINT") {
    data <- data %>%
      mutate(
        phase = case_when(
          week < T1 ~ "0_Pre-Policy",
          week >= T1 & week < T3 ~ "1_Policy-Active",
          week >= T3 ~ "2_Post-Reversal"
        ),
        phase = factor(phase, levels = c(
          "0_Pre-Policy", "1_Policy-Active", "2_Post-Reversal"
        ))
      )
  } else {
    data <- data %>%
      mutate(
        phase = case_when(
          week < T1 ~ "0_Pre-Policy",
          TRUE ~ "1_Policy-Active"
        ),
        phase = factor(phase, levels = c("0_Pre-Policy", "1_Policy-Active"))
      )
  }
  
  # Add election indicators
  data <- data %>%
    mutate(
      in_italian_election_window = week >= italian_election_window_start & 
                                    week <= italian_election_window_end,
      in_eu_election_window = week >= eu_election_window_start & 
                              week <= eu_election_window_end,
      in_any_election_window = in_italian_election_window | in_eu_election_window
    )
  
  return(data)
}

weekly_data_phased <- apply_phases(weekly_data, FINAL_BREAKPOINTS, MODEL_TYPE)
discovery_data_phased <- weekly_data_phased %>%
  filter(main_list == discovery_group)

# ============================================================================
# TABLE 5: REACH STATISTICS BY POLICY PHASE (Re-elected MPs)
# (Working Paper Table 5)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 5: REACH STATISTICS BY POLICY PHASE (", discovery_group, ")\n", sep = "")
cat("(Working Paper Table 5)\n")
cat(rep("=", 80), "\n\n", sep = "")

table5_stats <- discovery_data_phased %>%
  group_by(phase) %>%
  summarise(
    n_weeks = n(),
    mean_views = mean(avg_views, na.rm = TRUE),
    median_views = median(avg_views, na.rm = TRUE),
    sd_views = sd(avg_views, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  arrange(phase)

# Calculate change from previous phase
table5_stats <- table5_stats %>%
  mutate(
    change_pct = 100 * (mean_views - lag(mean_views)) / lag(mean_views)
  )

# Format Table 5
table5 <- table5_stats %>%
  transmute(
    Phase = phase,
    N_Weeks = n_weeks,
    Mean_Views = round(mean_views, 0),
    Median_Views = round(median_views, 0),
    SD = round(sd_views, 0),
    Change = ifelse(is.na(change_pct), "—", sprintf("%+.1f%%", change_pct))
  )

cat("TABLE 5 - Reach Statistics by Policy Phase:\n\n")
print(as.data.frame(table5), row.names = FALSE)
cat("\n")

write.csv(table5, "RQ1_Table5_reach_by_phase.csv", row.names = FALSE)
cat("Saved: RQ1_Table5_reach_by_phase.csv\n\n")

# ============================================================================
# TABLE 6: ENGAGEMENT METRICS BY POLICY PHASE (Re-elected MPs)
# (Working Paper Table 6)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 6: ENGAGEMENT METRICS BY POLICY PHASE (", discovery_group, ")\n", sep = "")
cat("(Working Paper Table 6)\n")
cat(rep("=", 80), "\n\n", sep = "")

table6_engagement <- discovery_data_phased %>%
  filter(!is.na(phase)) %>%
  group_by(phase) %>%
  summarise(
    Reactions_Mean = round(mean(avg_reactions, na.rm = TRUE), 1),
    Shares_Mean = round(mean(avg_shares, na.rm = TRUE), 1),
    Comments_Mean = round(mean(avg_comments, na.rm = TRUE), 1),
    .groups = "drop"
  ) %>%
  arrange(phase)

cat("TABLE 6 - Engagement Metrics by Phase:\n\n")
print(as.data.frame(table6_engagement), row.names = FALSE)
cat("\n")

write.csv(table6_engagement, "RQ1_Table6_engagement_by_phase.csv", row.names = FALSE)
cat("Saved: RQ1_Table6_engagement_by_phase.csv\n\n")

# ============================================================================
# TABLE 7: BREAKPOINT VALIDATION ACROSS GROUPS
# (Working Paper Table 7)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 7: BREAKPOINT VALIDATION ACROSS GROUPS\n")
cat("(Working Paper Table 7)\n")
cat(rep("=", 80), "\n\n", sep = "")

validation_results <- data.frame()

for (group in c(discovery_group, validation_groups)) {
  
  group_data <- weekly_data_phased %>%
    filter(main_list == group, !is.na(phase))
  
  if (nrow(group_data) == 0) next
  
  # Calculate phase stats
  group_phase_stats <- group_data %>%
    group_by(phase) %>%
    summarise(mean_views = mean(avg_views, na.rm = TRUE), .groups = "drop") %>%
    arrange(phase)
  
  # Calculate transitions
  group_means <- group_phase_stats$mean_views
  group_directions <- c()
  
  for (i in 2:length(group_means)) {
    direction <- ifelse(group_means[i] > group_means[i-1], "UP", "DOWN")
    group_directions <- c(group_directions, direction)
  }
  
  pattern <- paste(group_directions, collapse = " → ")
  
  # Check key transitions
  T1_correct <- length(group_directions) >= 1 && group_directions[1] == "DOWN"
  T3_correct <- length(group_directions) >= 1 && group_directions[length(group_directions)] == "UP"
  
  # Kruskal-Wallis test
  kw_test <- kruskal.test(avg_views ~ phase, data = group_data)
  
  validation_results <- rbind(validation_results, data.frame(
    Group = group,
    Pattern = pattern,
    T1_Valid = ifelse(T1_correct, "✓", "✗"),
    T3_Valid = ifelse(T3_correct, "✓", "✗"),
    KW_p = format.pval(kw_test$p.value, digits = 3),
    stringsAsFactors = FALSE
  ))
}

cat("TABLE 7 - Breakpoint Validation Across Groups:\n\n")
print(validation_results, row.names = FALSE)
cat("\n")

write.csv(validation_results, "RQ1_Table7_validation.csv", row.names = FALSE)
cat("Saved: RQ1_Table7_validation.csv\n\n")

# ============================================================================
# TABLE 8: CROSS-GROUP MAGNITUDE COMPARISON (VIEWS)
# (Working Paper Table 8)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 8: CROSS-GROUP MAGNITUDE COMPARISON (VIEWS)\n")
cat("(Working Paper Table 8)\n")
cat(rep("=", 80), "\n\n", sep = "")

table8_data <- data.frame()

for (group in unique(weekly_data_phased$main_list)) {
  
  group_stats <- weekly_data_phased %>%
    filter(main_list == group, !is.na(phase)) %>%
    group_by(phase) %>%
    summarise(mean_views = mean(avg_views, na.rm = TRUE), .groups = "drop") %>%
    arrange(phase)
  
  # Extract phase values
  phase0 <- group_stats$mean_views[group_stats$phase == "0_Pre-Policy"]
  phase1 <- group_stats$mean_views[group_stats$phase == "1_Policy-Active"]
  phase2 <- group_stats$mean_views[group_stats$phase == "2_Adjusted-Policy"]
  phase3 <- group_stats$mean_views[group_stats$phase == "3_Post-Reversal"]
  
  if (length(phase0) == 0) phase0 <- NA
  if (length(phase1) == 0) phase1 <- NA
  if (length(phase2) == 0) phase2 <- NA
  if (length(phase3) == 0) phase3 <- NA
  
  # Calculate delta (baseline to trough)
  trough <- min(c(phase1, phase2), na.rm = TRUE)
  if (is.infinite(trough)) trough <- phase1
  
  delta1 <- if (!is.na(phase0) && !is.na(trough)) {
    100 * (trough - phase0) / phase0
  } else NA
  
  table8_data <- rbind(table8_data, data.frame(
    Group = group,
    Phase_0 = round(phase0, 0),
    Phase_1 = round(phase1, 0),
    Phase_2 = round(phase2, 0),
    Phase_3 = round(phase3, 0),
    Delta = delta1,
    stringsAsFactors = FALSE
  ))
}

# Format for display
table8_display <- table8_data %>%
  mutate(
    Phase_0 = format(Phase_0, big.mark = ","),
    Phase_1 = format(Phase_1, big.mark = ","),
    Phase_2 = format(Phase_2, big.mark = ","),
    Phase_3 = format(Phase_3, big.mark = ","),
    Delta = sprintf("%.1f%%", Delta)
  )

cat("TABLE 8 - Cross-Group Magnitude Comparison:\n\n")
print(as.data.frame(table8_display), row.names = FALSE)
cat("\n")

write.csv(table8_data, "RQ1_Table8_cross_group.csv", row.names = FALSE)
cat("Saved: RQ1_Table8_cross_group.csv\n\n")

# ============================================================================
# TABLE 9: PAIRWISE PHASE COMPARISONS (DUNN'S TEST)
# (Working Paper Table 9)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 9: PAIRWISE PHASE COMPARISONS (DUNN'S TEST)\n")
cat("(Working Paper Table 9)\n")
cat(rep("=", 80), "\n\n", sep = "")

if (!require(dunn.test, quietly = TRUE)) {
  install.packages("dunn.test", repos = "https://cloud.r-project.org")
  library(dunn.test)
}

pairwise_results <- list()

for (group in unique(weekly_data_phased$main_list)) {
  group_data <- weekly_data_phased %>% filter(main_list == group, !is.na(phase))
  
  cat("===", group, "===\n")
  
  dunn_result <- dunn.test(group_data$avg_views, group_data$phase, 
                           method = "bonferroni", kw = FALSE, 
                           table = FALSE, list = TRUE)
  
  comparisons <- data.frame(
    comparison = dunn_result$comparisons,
    Z = round(dunn_result$Z, 3),
    p_adj = dunn_result$P.adjusted,
    sig = ifelse(dunn_result$P.adjusted < 0.001, "***",
                 ifelse(dunn_result$P.adjusted < 0.01, "**",
                        ifelse(dunn_result$P.adjusted < 0.05, "*", "n.s.")))
  )
  
  print(comparisons)
  cat("\n")
  
  pairwise_results[[group]] <- comparisons
}

# Create summary table
key_comparisons <- c("0_Pre-Policy - 1_Policy-Active",
                     "0_Pre-Policy - 2_Adjusted-Policy",
                     "1_Policy-Active - 2_Adjusted-Policy",
                     "2_Adjusted-Policy - 3_Post-Reversal",
                     "0_Pre-Policy - 3_Post-Reversal")

summary_matrix <- matrix(NA, nrow = length(key_comparisons), 
                         ncol = length(unique(weekly_data_phased$main_list)))
colnames(summary_matrix) <- unique(weekly_data_phased$main_list)
rownames(summary_matrix) <- c("Phase 0 vs 1", "Phase 0 vs 2", 
                               "Phase 1 vs 2", "Phase 2 vs 3", "Phase 0 vs 3")

for (i in seq_along(unique(weekly_data_phased$main_list))) {
  group <- unique(weekly_data_phased$main_list)[i]
  if (!is.null(pairwise_results[[group]])) {
    for (j in seq_along(key_comparisons)) {
      idx <- which(pairwise_results[[group]]$comparison == key_comparisons[j])
      if (length(idx) > 0) {
        summary_matrix[j, i] <- pairwise_results[[group]]$sig[idx]
      }
    }
  }
}

cat("\nTABLE 9 - Summary Matrix:\n\n")
print(summary_matrix)
cat("\n")

table9_df <- as.data.frame(summary_matrix)
table9_df$Comparison <- rownames(summary_matrix)
table9_df <- table9_df[, c("Comparison", colnames(summary_matrix))]
write.csv(table9_df, "RQ1_Table9_pairwise.csv", row.names = FALSE)
cat("Saved: RQ1_Table9_pairwise.csv\n\n")

# ============================================================================
# TABLE 10: PER-POST VS TOTAL WEEKLY REACH BY GROUP
# (Working Paper Table 10)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("TABLE 10: PER-POST VS TOTAL WEEKLY REACH BY GROUP\n")
cat("(Working Paper Table 10)\n")
cat(rep("=", 80), "\n\n", sep = "")

# Calculate total weekly reach
total_reach_weekly <- weekly_data_phased %>%
  filter(!is.na(phase)) %>%
  mutate(
    total_weekly_views = avg_views * n_posts
  )

# Summary by group and phase
total_reach_by_phase <- total_reach_weekly %>%
  group_by(main_list, phase) %>%
  summarise(
    mean_total_views = mean(total_weekly_views, na.rm = TRUE),
    .groups = "drop"
  )

# Create Table 10
table10_data <- data.frame()

for (group in unique(total_reach_by_phase$main_list)) {
  group_data <- total_reach_by_phase %>% filter(main_list == group)
  
  # Per-post metrics from table8
  perpost_phase0 <- table8_data$Phase_0[table8_data$Group == group]
  perpost_phase2 <- table8_data$Phase_2[table8_data$Group == group]
  
  # Total metrics
  total_phase0 <- group_data$mean_total_views[group_data$phase == "0_Pre-Policy"]
  total_phase2 <- group_data$mean_total_views[group_data$phase == "2_Adjusted-Policy"]
  
  if (length(total_phase0) == 0) total_phase0 <- NA
  if (length(total_phase2) == 0) total_phase2 <- NA
  
  # Calculate deltas
  perpost_delta <- if (!is.na(perpost_phase0) && !is.na(perpost_phase2) && perpost_phase0 > 0) {
    100 * (perpost_phase2 - perpost_phase0) / perpost_phase0
  } else NA
  
  total_delta <- if (!is.na(total_phase0) && !is.na(total_phase2) && total_phase0 > 0) {
    100 * (total_phase2 - total_phase0) / total_phase0
  } else NA
  
  table10_data <- rbind(table10_data, data.frame(
    Group = group,
    PerPost_Phase0 = round(perpost_phase0, 0),
    PerPost_Phase2 = round(perpost_phase2, 0),
    PerPost_Delta = perpost_delta,
    Total_Phase0 = round(total_phase0 / 1e6, 1),
    Total_Phase2 = round(total_phase2 / 1e6, 1),
    Total_Delta = total_delta,
    stringsAsFactors = FALSE
  ))
}

# Format for display
table10_display <- table10_data %>%
  mutate(
    PerPost_Phase0 = format(PerPost_Phase0, big.mark = ","),
    PerPost_Phase2 = format(PerPost_Phase2, big.mark = ","),
    PerPost_Delta = sprintf("%.1f%%", PerPost_Delta),
    Total_Phase0 = paste0(Total_Phase0, "M"),
    Total_Phase2 = paste0(Total_Phase2, "M"),
    Total_Delta = sprintf("%+.1f%%", Total_Delta)
  )

cat("TABLE 10 - Per-Post vs Total Weekly Reach:\n\n")
print(as.data.frame(table10_display), row.names = FALSE)
cat("\n")

write.csv(table10_data, "RQ1_Table10_perpost_vs_total.csv", row.names = FALSE)
cat("Saved: RQ1_Table10_perpost_vs_total.csv\n\n")

# ============================================================================
# FIGURE 1: TIME SERIES (ALL GROUPS)
# (Working Paper Figure 1)
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("CREATING FIGURES FOR WORKING PAPER\n")
cat(rep("=", 80), "\n\n", sep = "")

T1 <- FINAL_BREAKPOINTS$T1$date
T2 <- if (!is.null(FINAL_BREAKPOINTS$T2)) FINAL_BREAKPOINTS$T2$date else NULL
T3 <- if (!is.null(FINAL_BREAKPOINTS$T3)) FINAL_BREAKPOINTS$T3$date else NULL

# Figure 1: Main time series
figure1 <- ggplot(weekly_data, aes(x = week, y = avg_views, color = main_list)) +
  geom_line(linewidth = 0.8, alpha = 0.8) +
  geom_smooth(method = "loess", span = 0.2, se = FALSE, linewidth = 1.5) +
  geom_vline(xintercept = as.numeric(T1), linetype = "dashed", color = "red", linewidth = 1) +
  annotate("text", x = T1, y = Inf, label = "T₁: Policy\nImplementation", 
           vjust = 1.5, hjust = -0.1, color = "red", fontface = "bold", size = 3)

if (!is.null(T2)) {
  figure1 <- figure1 +
    geom_vline(xintercept = as.numeric(T2), linetype = "dashed", color = "orange", linewidth = 1) +
    annotate("text", x = T2, y = Inf, label = "T₂: Adjustment", 
             vjust = 1.5, hjust = -0.1, color = "orange", fontface = "bold", size = 3)
}

if (!is.null(T3)) {
  figure1 <- figure1 +
    geom_vline(xintercept = as.numeric(T3), linetype = "dashed", color = "darkgreen", linewidth = 1) +
    annotate("text", x = T3, y = Inf, label = "T₃: Policy\nReversal", 
             vjust = 1.5, hjust = -0.1, color = "darkgreen", fontface = "bold", size = 3)
}

figure1 <- figure1 +
  scale_y_continuous(labels = scales::comma) +
  scale_color_brewer(palette = "Set1") +
  labs(
    title = "RQ1: Meta's Political Content Policy Effects on Italian Political Actors",
    subtitle = paste0("Breakpoints: T₁ = ", as.character(T1),
                      if (!is.null(T2)) paste0(", T₂ = ", as.character(T2)) else "",
                      if (!is.null(T3)) paste0(", T₃ = ", as.character(T3)) else "",
                      " | Shaded regions indicate policy phases"),
    x = "Date",
    y = "Average Views per Post",
    color = "Group"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    plot.subtitle = element_text(size = 10, color = "gray40"),
    legend.position = "bottom"
  )

ggsave("RQ1_Figure1_timeseries.png", figure1, width = 14, height = 8, dpi = 300)
cat("Saved: RQ1_Figure1_timeseries.png (Working Paper Figure 1)\n")

# ============================================================================
# FIGURE 2: MPs REELECTED TRENDS
# (Working Paper Figure 2)
# ============================================================================

discovery_data_plot <- weekly_data_phased %>% filter(main_list == discovery_group)

# Calculate phase means for annotation
phase_means <- discovery_data_plot %>%
  group_by(phase) %>%
  summarise(
    mean_views = mean(avg_views, na.rm = TRUE),
    mid_date = median(week),
    .groups = "drop"
  )

figure2 <- ggplot(discovery_data_plot, aes(x = week, y = avg_views)) +
  geom_line(color = "steelblue", linewidth = 0.6, alpha = 0.7) +
  geom_smooth(method = "loess", span = 0.15, se = TRUE, 
              color = "navy", fill = "lightblue", linewidth = 1.2) +
  geom_vline(xintercept = as.numeric(T1), linetype = "dashed", color = "red", linewidth = 1)

if (!is.null(T2)) {
  figure2 <- figure2 +
    geom_vline(xintercept = as.numeric(T2), linetype = "dashed", color = "orange", linewidth = 1)
}

if (!is.null(T3)) {
  figure2 <- figure2 +
    geom_vline(xintercept = as.numeric(T3), linetype = "dashed", color = "darkgreen", linewidth = 1)
}

# Add phase mean annotations
for (i in 1:nrow(phase_means)) {
  figure2 <- figure2 + 
    annotate("text", 
             x = phase_means$mid_date[i], 
             y = max(discovery_data_plot$avg_views, na.rm = TRUE) * 0.95,
             label = paste0("Mean: ", format(round(phase_means$mean_views[i], 0), big.mark = ",")),
             color = "gray30", size = 3, fontface = "italic")
}

figure2 <- figure2 +
  scale_y_continuous(labels = scales::comma) +
  labs(
    title = paste0("RQ1: Reach Trends for ", discovery_group),
    subtitle = paste0("T₁ = ", as.character(T1), 
                      if (!is.null(T2)) paste0(" | T₂ = ", as.character(T2)) else "",
                      if (!is.null(T3)) paste0(" | T₃ = ", as.character(T3)) else "",
                      "\nShaded: Gray = Pre-Policy, Red = Policy Active, Green = Post-Reversal"),
    x = "Date",
    y = "Average Views per Post"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", size = 14)
  )

ggsave("RQ1_Figure2_discovery.png", figure2, width = 12, height = 6, dpi = 300)
cat("Saved: RQ1_Figure2_discovery.png (Working Paper Figure 2)\n")

# ============================================================================
# FIGURE 3: INDIVIDUAL GROUP TRENDS (FACETED)
# (Working Paper Figure 3)
# ============================================================================

figure3 <- ggplot(weekly_data_phased, aes(x = week, y = avg_views)) +
  geom_line(color = "steelblue", linewidth = 0.5, alpha = 0.6) +
  geom_smooth(method = "loess", span = 0.2, se = FALSE, color = "navy", linewidth = 1) +
  geom_vline(xintercept = as.numeric(T1), linetype = "dashed", color = "red", linewidth = 0.8)

if (!is.null(T2)) {
  figure3 <- figure3 +
    geom_vline(xintercept = as.numeric(T2), linetype = "dashed", color = "orange", linewidth = 0.8)
}

if (!is.null(T3)) {
  figure3 <- figure3 +
    geom_vline(xintercept = as.numeric(T3), linetype = "dashed", color = "darkgreen", linewidth = 0.8)
}

figure3 <- figure3 +
  facet_wrap(~main_list, scales = "free_y", ncol = 2) +
  scale_y_continuous(labels = scales::comma) +
  labs(
    title = "RQ1: Reach Trends by Group (Breakpoint Validation)",
    subtitle = paste0("T₁ = ", as.character(T1), " (red)",
                      if (!is.null(T2)) paste0(" | T₂ = ", as.character(T2), " (orange)") else "",
                      if (!is.null(T3)) paste0(" | T₃ = ", as.character(T3), " (green)") else ""),
    x = "Date",
    y = "Average Views per Post"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold"),
    strip.text = element_text(face = "bold")
  )

ggsave("RQ1_Figure3_faceted.png", figure3, width = 12, height = 10, dpi = 300)
cat("Saved: RQ1_Figure3_faceted.png (Working Paper Figure 3)\n\n")

# ============================================================================
# SUMMARY OF KEY FINDINGS
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("SUMMARY OF KEY FINDINGS FOR WORKING PAPER\n")
cat(rep("=", 80), "\n\n", sep = "")

# Calculate key statistics
baseline_mean <- table5_stats$mean_views[1]
trough_mean <- min(table5_stats$mean_views[table5_stats$phase %in% c("1_Policy-Active", "2_Adjusted-Policy")], na.rm = TRUE)
post_reversal_mean <- table5_stats$mean_views[nrow(table5_stats)]

decline_from_baseline <- 100 * (trough_mean - baseline_mean) / baseline_mean
recovery_pct_of_baseline <- 100 * post_reversal_mean / baseline_mean

cat("KEY STATISTICS FOR", discovery_group, ":\n\n")
cat("  Baseline (Pre-Policy) Mean:", format(round(baseline_mean, 0), big.mark = ","), "views/post\n")
cat("  Trough (During Policy) Mean:", format(round(trough_mean, 0), big.mark = ","), "views/post\n")
cat("  Post-Reversal Mean:", format(round(post_reversal_mean, 0), big.mark = ","), "views/post\n\n")
cat("  Decline from Baseline:", sprintf("%.1f%%", decline_from_baseline), "\n")
cat("  Recovery as % of Baseline:", sprintf("%.1f%%", recovery_pct_of_baseline), "\n\n")

cat("BREAKPOINT SUMMARY:\n")
cat("  T1 (Implementation):", as.character(T1), "\n")
if (!is.null(T2)) cat("  T2 (Adjustment):", as.character(T2), "\n")
if (!is.null(T3)) cat("  T3 (Reversal):", as.character(T3), "\n")
cat("\n")

# ============================================================================
# OUTPUT FILES SUMMARY
# ============================================================================

cat("\n")
cat(rep("=", 80), "\n", sep = "")
cat("OUTPUT FILES PRODUCED (Aligned with Working Paper)\n")
cat(rep("=", 80), "\n\n", sep = "")

cat("TABLES:\n")
cat("  • RQ1_Table2_account_post_counts.csv  → Working Paper Table 2\n")
cat("  • RQ1_Table3_engagement_stats.csv     → Working Paper Table 3\n")
cat("  • RQ1_Table4_breakpoints.csv          → Working Paper Table 4\n")
cat("  • RQ1_Table5_reach_by_phase.csv       → Working Paper Table 5\n")
cat("  • RQ1_Table6_engagement_by_phase.csv  → Working Paper Table 6\n")
cat("  • RQ1_Table7_validation.csv           → Working Paper Table 7\n")
cat("  • RQ1_Table8_cross_group.csv          → Working Paper Table 8\n")
cat("  • RQ1_Table9_pairwise.csv             → Working Paper Table 9\n")
cat("  • RQ1_Table10_perpost_vs_total.csv    → Working Paper Table 10\n\n")

cat("FIGURES:\n")
cat("  • RQ1_Figure1_timeseries.png  → Working Paper Figure 1\n")
cat("  • RQ1_Figure2_discovery.png   → Working Paper Figure 2\n")
cat("  • RQ1_Figure3_faceted.png     → Working Paper Figure 3\n\n")

# Save complete results object
results_summary <- list(
  analysis_date = Sys.time(),
  data_file = weekly_file,
  model_type = MODEL_TYPE,
  discovery_group = discovery_group,
  validation_groups = validation_groups,
  breakpoints = FINAL_BREAKPOINTS,
  table2 = table2,
  table3 = table3_stats,
  table4 = table4_data,
  table5 = table5_stats,
  table6 = table6_engagement,
  table7 = validation_results,
  table8 = table8_data,
  table9 = table9_df,
  table10 = table10_data,
  key_findings = list(
    baseline_mean = baseline_mean,
    trough_mean = trough_mean,
    post_reversal_mean = post_reversal_mean,
    decline_pct = decline_from_baseline,
    recovery_pct_of_baseline = recovery_pct_of_baseline
  )
)

saveRDS(results_summary, "RQ1_results_summary.rds")
cat("Complete results saved to: RQ1_results_summary.rds\n")

cat("\n=== RQ1 Analysis Complete ===\n")

[NOTICE] 8 output(s) filtered out